# Librerías

Importamos las librerías necesarias y cargamos variables de entorno

In [12]:
%pip install pymongo
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from apis.mongodb_api import MongoAPI
from apis.messageData import MessageData
from apis.relationData import make_rel_id
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv(ROOT / ".env.local")

MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB = os.getenv("MONGO_DB")
api = MongoAPI(MONGO_URI, MONGO_DB)
# Reset inicial
api.clear_messages()
api.clear_snapshots()

2

# Operaciones de Mensajes

# Insertar Mensaje

insert_message(dict) inserta un mensaje en la colección *messages*. Devuelve el id del mensaje insertado

In [3]:
message = MessageData(
    msg_id="msg001",
    sender_id="u6",
    receiver_id="u7",
    rel_id=make_rel_id("u6", "u7"),
    content="Hello, u6!"
)
id = api.insert_message(message.to_dict())

# Obtener un mensaje por _id o msg_id

get_msg_by_id(_id) o get_msg_by_msg_id(msg_id) permiten obtener el documento a partir de su id (generado por mongo) o msg_id (id unico)

In [4]:
api.get_message_by_msg_id("msg001")

{'_id': ObjectId('6963e243e05e2d3bcead4d62'),
 'sender_id': 'u6',
 'receiver_id': 'u7',
 'content': 'Hello, u6!',
 'rel_id': 'u6-u7',
 'msg_id': 'msg001',
 'timestamp': datetime.datetime(2026, 1, 11, 17, 47, 47, 651000),
 'metadata': {}}

In [5]:
api.get_message_by_id(id)

{'_id': ObjectId('6963e243e05e2d3bcead4d62'),
 'sender_id': 'u6',
 'receiver_id': 'u7',
 'content': 'Hello, u6!',
 'rel_id': 'u6-u7',
 'msg_id': 'msg001',
 'timestamp': datetime.datetime(2026, 1, 11, 17, 47, 47, 651000),
 'metadata': {}}

# Lista de mensajes
list_messages(limit, newest_first) muestra todos los mensajes con un limite de forma ascendente o descendente, dependiendo del segundo parametro (newest_first)

In [6]:
api.list_messages()

[{'_id': ObjectId('6963e243e05e2d3bcead4d62'),
  'sender_id': 'u6',
  'receiver_id': 'u7',
  'content': 'Hello, u6!',
  'rel_id': 'u6-u7',
  'msg_id': 'msg001',
  'timestamp': datetime.datetime(2026, 1, 11, 17, 47, 47, 651000),
  'metadata': {}}]

# Borrar mensaje
delete_msg_by_msg_id(msg_id) borra el mensaje por su msg_id. Devuelve 0 si no logra eliminarlo o 1 en caso de hacerlo.

In [7]:
api.delete_message_by_msg_id("msg001")
api.list_messages()

[]

# Consultas

# Contar mensajes en un tiempo para cierta relación
count_messages_by_time_for_relation(rel_id, time, start, end) cuenta los mensajes para una relación en un cierto periodo. Time solo puede tomar los valores *'hour'* *'day'* *'year'*, de lo contrario lanza excepción. Start y end indican el inicio y final del periodo

In [8]:
m1 = MessageData(
    sender_id="u1",
    receiver_id="u2",
    rel_id=make_rel_id("u1", "u2"),
    content="Hola u2"
)

m2 = MessageData(
    sender_id="u1",
    receiver_id="u2",
    rel_id=make_rel_id("u1", "u2"),
    content="¿Qué tal estás?"
)

m3 = MessageData(
    sender_id="u1",
    receiver_id="u2",
    rel_id=make_rel_id("u1", "u2"),
    content="¿Vienes hoy a clase?"
)

messages = [m1, m2, m3]
for m in messages:
    api.insert_message(m.to_dict())

In [9]:
api.count_messages_by_time_for_relation(make_rel_id("u1", "u2"), time="day", start=datetime(2024, 1, 1), end=datetime(2026, 12, 31))

[{'count': 3, 'periodo': '2026-01-11'}]

# Contar mensajes por usuario
count_messages_by_user(user_id, role) cuenta los mensajes enviados por un usuario. El rol del usuario puede ser *'sender'* *'receiver'* *'either'*, de lo contrario lanza excepción

In [10]:
api.count_messages_by_user("u1")

3

In [11]:
api.close()